***Color manual***

<div class="alert alert-block alert-success">
<b>Green - Libraries</b>
</div>

<div class="alert alert-block alert-info">
<b>Blue - Information</b>
</div>

<div class="alert alert-block alert-danger">
<b>Red - Problems</b>
</div>

***

<div class="alert alert-block alert-success">
<b>Import libraries</b>
</div>

In [1]:
import numpy as np
import chart_studio
import pandas as pd
import statsmodels.api as sm
import plotly.graph_objs as go
import chart_studio.plotly as py

from pandas_datareader import data

chart_studio.tools.set_config_file(world_readable = True, sharing = 'public')

C:\Users\Sif\Anaconda3\lib\site-packages\pandas_datareader\compat\__init__.py:7: FutureWarning:

pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.



<div class="alert alert-block alert-info">
<b>Load the stock historical data, transform in montly data and get percentage return</b>
</div>

In [2]:
R = data.DataReader('SNE', 'yahoo', start = '2017/01/01', end = '2019/12/31')
R = R.resample('M').last().pct_change()
R.head()

,High,Low,Open,Close,Volume,Adj Close
Date,,,,,,
2017-01-31,NaN,NaN,NaN,NaN,NaN,NaN
2017-02-28,0.025008,0.026246,0.025777,0.022795,-0.227816,0.022795
2017-03-31,0.084430,0.087407,0.086018,0.089470,-0.217370,0.092539
2017-04-30,0.023979,0.017267,0.018985,0.023421,0.509737,0.023421
2017-05-31,0.060133,0.066725,0.066376,0.060834,0.071867,0.060834


<div class="alert alert-block alert-info">
<b>Percentage return of the market index S&P 500 with GSPC</b>
</div>

In [3]:
Rm = data.DataReader('^GSPC', 'yahoo', start = '2017/01/01', end = '2019/12/31')
Rm = Rm.resample('M').last().pct_change()
Rm.head()

,High,Low,Open,Close,Volume,Adj Close
Date,,,,,,
2017-01-31,NaN,NaN,NaN,NaN,NaN,NaN
2017-02-28,0.038919,0.040468,0.040483,0.037198,0.030016,0.037198
2017-03-31,0.001081,0.001543,-0.000533,-0.000389,-0.203326,-0.000389
2017-04-30,0.009842,0.008364,0.012204,0.009091,0.108571,0.009091
2017-05-31,0.009320,0.008911,0.009170,0.011576,0.214573,0.011576


<div class="alert alert-block alert-info">
<b>Percentage return of risk free investment with DGS10 = One Month Treasury Constant Maturity Rate</b>
</div>

In [4]:
Rf = data.DataReader('DGS1MO', 'fred', start = '2017/02/01', end = '2019/12/31')
Rf = Rf.resample('M').last()
Rf = Rf/100
Rf.head()

,DGS1MO
DATE,
2017-02-28,0.0040
2017-03-31,0.0074
2017-04-30,0.0068
2017-05-31,0.0086
2017-06-30,0.0084


<div class="alert alert-block alert-info">
<b>Table for Capital Asset Pricing Model</b>
</div>

In [5]:
df = pd.DataFrame({'R': R['Adj Close'], 'Rm': Rm['Adj Close'], 'Rf': Rf['DGS1MO']}).dropna()
df.head()

,R,Rm,Rf
Date,,,
2017-02-28,0.022795,0.037198,0.0040
2017-03-31,0.092539,-0.000389,0.0074
2017-04-30,0.023421,0.009091,0.0068
2017-05-31,0.060834,0.011576,0.0086
2017-06-30,0.042873,0.004814,0.0084


<div class="alert alert-block alert-info">
<b>Linear regression (CAPM) for expected returns</b>
</div>

In [6]:
x = df['Rm']
y = df['R']

x = sm.add_constant(x)
model = sm.OLS(y, x, missing = 'drop')
results = model.fit()
results.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                      R   R-squared:                       0.259
Model:                            OLS   Adj. R-squared:                  0.236
Method:                 Least Squares   F-statistic:                     11.53
Date:                Sat, 09 May 2020   Prob (F-statistic):            0.00180
Time:                        23:57:21   Log-Likelihood:                 49.198
No. Observations:                  35   AIC:                            -94.40
Df Residuals:                      33   BIC:                            -91.28
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const          0.0156      0.011      1.443      0.159      -0.006       0.038
Rm             1.0042      0.296      3.395      0.002       0.402       1.606
==============================================================================
Omnibus:                        0.707   Durbin-Watson:                   2.104
Prob(Omnibus):                  0.702   Jarque-Bera (JB):                0.107
Skew:                           0.020   Prob(JB):                        0.948
Kurtosis:                       3.268   Cond. No.                         28.6
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

<div class="alert alert-block alert-info">
<b>Add deviation from the mean, predicted values, simple residuals, squared residuals and standard error of estimate to table</b>
</div>

In [7]:
# Predicted values for excess return of stock(R - Rf)
y_pred = results.predict()
df['Predicted expected returns'] = y_pred

# Simple residuals
residual = results.resid
df['Residuals'] = results.resid

# Squared residuals
squared_res = df['Residuals']**2
df['Squared residuals'] =  squared_res

# Sum of squared residuals
sum_squared_res = df['Squared residuals'].sum()
print('Sum of Squared residual: %f' %sum_squared_res)

# Squared error of estimated
squared_error = results.bse[1]
print('Standard errors: %f' %squared_error)

df.head()

Sum of Squared residual: 0.123214
Standard errors: 0.295782


,R,Rm,Rf,Predicted expected returns,Residuals,Squared residuals
Date,,,,,,
2017-02-28,0.022795,0.037198,0.0040,0.052932,-0.030137,0.000908
2017-03-31,0.092539,-0.000389,0.0074,0.015185,0.077354,0.005984
2017-04-30,0.023421,0.009091,0.0068,0.024705,-0.001284,0.000002
2017-05-31,0.060834,0.011576,0.0086,0.027201,0.033633,0.001131
2017-06-30,0.042873,0.004814,0.0084,0.020410,0.022463,0.000505


<div class="alert alert-block alert-info">
<b>Expected monthly % returns plot</b>
</div>

In [8]:
fig = go.Figure()

fig.add_trace(go.Scatter(x = df['Rm'], y = df['R'], 
    name = 'Observed expected returns', 
    mode = 'markers',
    marker = dict(size = 8),
    hoverinfo = 'x+y+text',
    line = dict(color = 'rgb(66, 133, 244)')))

fig.add_trace(go.Scatter(x = df['Rm'], y = df['Predicted expected returns'],
    error_y = dict(value = squared_error*100, 
    color = 'rgb(52, 168, 83)'),
    name = 'Predicted expected returns',
    mode = 'lines',
    marker = dict(size = 8),
    hoverinfo = 'x+y+text',
    line = dict(color = 'rgb(234, 67, 53)')))

fig.update_layout(title_text = 'Sony (SNE) and S&P 500 Index from 2017 to 2019', 
    title_x = 0.5,
    title_y = 0.8,
    autosize = True,
    legend = dict(
    x = 0, y = 1.5),
    xaxis_title = 'Monthly market expected return (%)',
    yaxis_title = 'Monthly SNE expected return (%)',
    paper_bgcolor = 'rgba(1,1,1,1)',
    plot_bgcolor = 'rgba(1,1,1,1)',
    hoverlabel = dict(
    font_size = 15, 
    font_family = 'Helvetica'),
    font = dict(
    family = 'Helvetica, Helvetica',
    size = 15,
    color = 'white'))

fig.update_xaxes(ticks = 'outside', 
                 tickwidth = 2, 
                 tickcolor = 'black', 
                 ticklen = 10,
                 showgrid = False,
                 zeroline = False)

fig.update_yaxes(ticks = 'outside', 
                 tickwidth = 2, 
                 tickcolor = 'black', 
                 ticklen = 10,
                 showgrid = False,
                 zeroline = False)

py.iplot(fig, filename = 'Linear regression of monthly expected return - Sony (SNE) and S&P 500', auto_open = False)